In [3]:
import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
import os
from natsort import natsorted
from matplotlib import pyplot as plt
from skimage.restoration import inpaint
import shutil
import elasticdeform
import random

노트북 설명

목적  : 포토샵으로 세그멘테이션한 이미지를 불러와, 원본의 이미지에서 세그멘테이션한 부분을 추출하는 코드.

2023.03.10

이미지 상에서 나타나는 결함은 GrayScale값의 변화로 나타난다. 결함 부분에서 결함이 없을 때의 GrayScale값과 결함이 있을 때의 GrayScale값의 차이를 이용하여 결함을 추출하려 한다.

inpaint 관련 여러 알고리즘 또는 모델을 알아보려 함.

In [92]:
def extract_flaw(ground_truth, origin, cleared, padding=0, file_name=None, category=None):
    """
    Extract flaw from origin
    :param ground_truth: ground truth image
    :param origin: origin image
    :param padding: padding size
    :return: flaw image
    """
    image_name = os.path.basename(ground_truth).split('_')[0]
    # Read ground truth
    mask = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE) # 우리가 만든 마스킹
    # Read origin
    image_orig = cv2.imread(origin, cv2.IMREAD_GRAYSCALE) # 원본 이미지(결함 있음)
    #float32로 변환
    image_orig = np.float32(image_orig)
    image_result = cv2.imread(cleared, cv2.IMREAD_GRAYSCALE) # 원본 이미지(결함 없음)
    #float32로 변환
    image_diff = np.subtract(image_orig, image_result) # 원본 이미지 - 결함 없는 이미지 = 결함 이미지
    print(image_diff.shape, np.min(image_diff), np.max(image_diff))
    
    #결과 이미지 : image_result
    #마스크 이미지 : image_defect
    #원본 이미지 : image_orig
    
    #마스크 확장. 
    _, gt_bin = cv2.threshold(mask, 0, 255, cv2.THRESH_OTSU)  # 0보다 크면 255, 0보다 작으면 0 
    dilatation = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    gt_bin_dil = cv2.dilate(gt_bin, dilatation, iterations=1) # 팽창. 마스킹을 좀 더 키워서 결함을 더 잘 감싸도록 함.
    
    #image_diff = cv2.GaussianBlur(image_diff, (3,3), 0) # 가우시안 블러 적용. 노이즈 제거
    
    
    cnt, labels, stats, centroids = cv2.connectedComponentsWithStats(gt_bin_dil) #윤곽선 추출해서 사각형으로 보여줌.
    count = 0
    for i in range(1, cnt):

        x, y, w, h, s = stats[i]
        x = x - padding 
        y = y - padding
        w = w + 2 * padding
        h = h + 2 * padding
        if x < 0:
            x = 0
        if y < 0:
            y = 0
        if x + w > mask.shape[1]:
            w = mask.shape[1] - x
        if y + h > mask.shape[0]:
            h = mask.shape[0] - y
        flaw = image_diff[y:y + h, x:x + w]
        
        
        mask_patch = mask[y:y + h, x:x + w]
        # numpy array로 저장
        mask_patch = cv2.GaussianBlur(mask_patch, (51,51), 0)
        mask_patch = np.float32(mask_patch)
        mask_patch = cv2.normalize(mask_patch, None, 0, 1, cv2.NORM_MINMAX)
        # mask_patch는 결함이 있는 부분만 흰색, 나머지는 검은색으로 표시된 이미지
        # 가우시안 블러를 사용해 결함 경계를 흐리게 함

        
        flaw = flaw * mask_patch
        # 실제 결함 패치인 flaw에 mask_patch를 곱해 결함 경계에 있는 픽셀들을 흐리게 함

        np.save(f'/home/dais01/VF_MASTER/Data/{category}/Extracted_Flaw/{image_name}_{count}.npy', flaw)
        
        #np.save(f'/home/dais01/VF_MASTER/Data/flaw/Extracted_Flaw/{image_name}_{count}_mask.npy', mask_patch)
        flaw = cv2.normalize(flaw, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(f'/home/dais01/VF_MASTER/Data/{category}/Extracted_Flaw/{image_name}_{count}.png', flaw, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        count += 1
        


In [93]:
category = "PO"
image_list = glob(f"/home/dais01/VF_MASTER/Data/{category}/Mask/*.png")
print(len(image_list))
for image in image_list:
    image_name = image.split('/')[-1].split('_')[0]
    mask_image = f"/home/dais01/VF_MASTER/Data/{category}/Mask/{image_name}_mask001.png"
    cleared_image = f"/home/dais01/VF_MASTER/Data/{category}/Inpainted_to_Raw/{image_name}.png"
    origin_image = f"/home/dais01/VF_MASTER/Data/{category}/Raw_Image/{image_name}.png"
    
    extract_flaw(mask_image, origin_image, cleared_image, padding=10, file_name=image_name, category=category)

58
(1256, 1256) -2.0 4.0
(1256, 1256) -3.0 5.0
(1256, 1256) -4.0 35.0
(1256, 1256) -2.0 12.0
(1256, 1256) -2.0 7.0
(1256, 1256) -3.0 4.0
(1256, 1256) -3.0 17.0
(1256, 1256) -2.0 3.0
(1256, 1256) -2.0 4.0
(1256, 1256) -10.0 30.0
(1256, 1256) -3.0 6.0
(1256, 1256) -3.0 5.0
(1256, 1256) -3.0 29.0
(1256, 1256) -2.0 5.0
(1256, 1256) -3.0 20.0
(1256, 1256) -3.0 8.0
(1256, 1256) -2.0 4.0
(1256, 1256) -3.0 23.0
(1256, 1256) -2.0 4.0
(1256, 1256) -3.0 14.0
(1256, 1256) -2.0 11.0
(1256, 1256) -3.0 5.0
(1256, 1256) -2.0 11.0
(1256, 1256) -3.0 26.0
(1256, 1256) -2.0 3.0
(1256, 1256) -3.0 3.0
(1256, 1256) -2.0 23.0
(1256, 1256) -3.0 5.0
(1256, 1256) -2.0 3.0
(1256, 1256) -3.0 19.0
(1256, 1256) -2.0 6.0
(1256, 1256) -2.0 6.0
(1256, 1256) -53.0 34.0
(1256, 1256) -2.0 14.0
(1256, 1256) -3.0 9.0
(1256, 1256) -2.0 6.0
(1256, 1256) -3.0 4.0
(1256, 1256) -2.0 16.0
(1256, 1256) -3.0 9.0
(1256, 1256) -2.0 3.0
(1256, 1256) -3.0 20.0
(1256, 1256) -3.0 6.0
(1256, 1256) -3.0 6.0
(1256, 1256) -3.0 5.0
(1256, 125

In [94]:

#이미지의 외곽 padding pixel 부분에 np.linspace를 이용하여 0~1까지의 값을 넣어준다.
def padding_image(npy, padding):
    img = np.load(npy)
    mask_img1 = np.ones((img.shape[0], img.shape[1]))
    mask_img1[:padding, :] = np.linspace(0, 1, padding).repeat(img.shape[1]).reshape(padding, -1)
    mask_img1[-padding:, :] = np.linspace(1, 0, padding).repeat(img.shape[1]).reshape(padding, -1)
    
    mask_img2 = np.ones((img.shape[0], img.shape[1]))
    mask_img2[:, :padding] = np.linspace(0, 1, padding).reshape(-1, 1).repeat(img.shape[0], axis=1).T
    mask_img2[:, -padding:] = np.linspace(1, 0, padding).reshape(-1, 1).repeat(img.shape[0], axis=1).T
    
    mask_img = mask_img1 * mask_img2
    img = img * mask_img
    
    img_cv = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    img_cv = np.uint8(img_cv)
    cv2.imwrite(npy.replace('.npy', '.png'), img_cv)
    np.save(npy, img)
    




In [95]:
flaw_list = glob(f"/home/dais01/VF_MASTER/Data/{category}/Extracted_Flaw/*.npy")
for flaw in flaw_list:
    padding_image(flaw, 10)

In [8]:
flaw_list = glob(f"/home/dais01/VF_MASTER/Data/PO/Extracted_Flaw/*.npy")
for flaw in flaw_list:
    img = np.load(flaw)
    for j in range(5):
        sigma = random.randint(1, 4)
        points = random.randint(1, 3)
        img_deformed = elasticdeform.deform_random_grid(img, sigma=sigma, points=points, order=1)
        np.save(flaw.replace('.npy', f'_deformed_{j}.npy').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)
        img_deformed = cv2.normalize(img_deformed, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(flaw.replace('.npy', f'_deformed_{j}.png').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)

In [10]:
flaw_list = glob(f"/home/dais01/VF_MASTER/Data/CT/Extracted_Flaw/*.npy")
for flaw in flaw_list:
    img = np.load(flaw)
    for j in range(5):
        sigma = random.randint(4, 7)
        points = random.randint(3, 5)
        img_deformed = elasticdeform.deform_random_grid(img, sigma=sigma, points=points, order=1)
        np.save(flaw.replace('.npy', f'_deformed_{j}.npy').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)
        img_deformed = cv2.normalize(img_deformed, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(flaw.replace('.npy', f'_deformed_{j}.png').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)

In [11]:
flaw_list = glob(f"/home/dais01/VF_MASTER/Data/Scratch/Extracted_Flaw/*.npy")
for flaw in flaw_list:
    img = np.load(flaw)
    for j in range(5):
        sigma = random.randint(4, 7)
        points = random.randint(3, 5)
        img_deformed = elasticdeform.deform_random_grid(img, sigma=sigma, points=points, order=1)
        np.save(flaw.replace('.npy', f'_deformed_{j}.npy').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)
        img_deformed = cv2.normalize(img_deformed, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(flaw.replace('.npy', f'_deformed_{j}.png').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)

In [12]:
flaw_list = glob(f"/home/dais01/VF_MASTER/Data/Leftover/Extracted_Flaw/*.npy")
for flaw in flaw_list:
    img = np.load(flaw)
    for j in range(5):
        sigma = random.randint(3, 5)
        points = random.randint(2, 4)
        img_deformed = elasticdeform.deform_random_grid(img, sigma=sigma, points=points, order=1)
        np.save(flaw.replace('.npy', f'_deformed_{j}.npy').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)
        img_deformed = cv2.normalize(img_deformed, None, 0, 255, cv2.NORM_MINMAX)
        cv2.imwrite(flaw.replace('.npy', f'_deformed_{j}.png').replace("Extracted_Flaw", "Deformed_Flaw"), img_deformed)